In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('CoinMarketCap all coins.csv', parse_dates=['timestamp', 'timeOpen', 'timeClose', 'timeHigh', 'timeLow'])

In [ ]:
def find_high_low_values_for_coins(data):
    results = []

    # Group the data by 'symbol' and iterate through each group
    for symbol, group in data.groupby('symbol'):
        highest_high_row = group.loc[group['high'].idxmax()]

        # Find the rows after the highest high within this group
        rows_after_highest = group[group['timeLow'] > highest_high_row['timeHigh']]

        # If there are rows after the highest high, find the lowest 'low'
        if not rows_after_highest.empty:
            lowest_low_row = rows_after_highest.loc[rows_after_highest['low'].idxmin()]
            first_price = group.iloc[0]
            last_price = group.iloc[-1]
            results.append({
                'symbol': symbol,
                'highest_high_value': highest_high_row['high'],
                'highest_high_date': highest_high_row['timeHigh'],
                'lowest_low_value_after_highest': lowest_low_row['low'],
                'lowest_low_date_after_highest': lowest_low_row['timeLow'],
                'first_known_price': first_price['open'],
                'first_known_date': first_price['timeOpen'],
                'last_known_price': last_price['close'],
                'last_known_date': last_price['timeClose']
            })

    return results

result_per_coin = find_high_low_values_for_coins(data)



In [ ]:
x = pd.DataFrame(result_per_coin)

x.rename(columns={'symbol':'name', 
                  'highest_high_value':'high',
                  'highest_high_date':'hdate',
                  'lowest_low_value_after_highest':'low',
                  'lowest_low_date_after_highest':'ldate',
                  'first_known_price':'first_price',
                  'first_known_date':'first_date',
                  'last_known_price':'last_price',
                  'last_known_date':'last_date'}, inplace=True)

x = x[(x['high'] > 0) & (x['low'] > 0)]

x = x[~(x['low'] >= x['last_price'])]

x.reset_index(inplace=True, drop=True)

In [ ]:
x[(x['last_date'] - x['first_date']) < pd.Timedelta('30 days')]

In [ ]:
h = pd.DataFrame()

In [ ]:
h['name'] = x['name']

In [ ]:
h['HL_delta'] = (x['high'] - x['low'])/x['high']

In [ ]:
h['LC_delta'] = (x['last_price'] - x['low'])/x['low']

In [ ]:
h['HL_time_delta'] = x['ldate'] - x['hdate']

In [ ]:
h['LC_time_delta'] = x['last_date'] - x['ldate']

In [ ]:
h.to_csv('HL_LC_data.csv', index_label='Id')